# RRSG 2019 Challenge 

<div class="alert alert-info">
  <strong><span class="badge">BLUE BOX</span></strong> <b> Steps for RRSG 2019 challenge tasks</b> 
</div>

<div class="alert alert-warning">
  <strong><span class="badge">YELLOW BOX</span></strong> <b> Optional steps for demonstration </b> 
</div>

<div class="alert alert-danger">
  <strong><span class="badge">RED BOX</span></strong> <b> Warnings to the user</b> 
</div>


In [ ]:
% Load octave packages 
pkg load image 
pkg load optim

In [ ]:
% Mex c files for gridding by Brian Hargreaves and Philip Beatty 
% http://mrsrl.stanford.edu/~brian/gridding/

mex gridlut_mex.c
mex calcdcflut_mex.c

## 1. Load ISMRM RRSG 2019 challenge data.

<div class="alert alert-info">
  <strong><span class="badge">1.1</span></strong> <b>Please see this <a href="https://osf.io/xne9w/">OSF page</a> for details.</b> 
</div>

Data is downloaded from the OSF and added to the portable software environment using [`postBuild`](postBuild) configuration file.


In [ ]:
load('/tmp/rrsg_challenge/brain_radial_96proj_12ch.mat');
whos % Show variables in the current scope 

<div class="alert alert-info">
  <strong><span class="badge">1.2</span></strong> <b>Change data order to follow BART's dimension convention</b> 
</div>

<div class="alert alert-danger">
  <strong><span class="badge">Warning</span></strong> <b>Do not run the following cell more than once after loading data (previous cell).</b><br> Otherwise, data will be permutted once again and won't be following BART's convention anymore.

--> You can refer to [this documentation](https://buildmedia.readthedocs.org/media/pdf/bart-doc/latest/bart-doc.pdf) for BART's dimension conventions.

</div>


In [ ]:
rawdata = permute(rawdata,[4,3,2,1]); 
trajectory = permute(trajectory,[3,2,1]);
[~,nFE,nSpokes,nCh] = size(rawdata);
whos

<div class="alert alert-warning">
  <strong><span class="badge">1.3</span></strong> <b>Display raw data from each channel.</b> 
</div>

In [ ]:
for ii=1:12
cc = squeeze((rawdata(1,:,:,ii)));
% Brighten convention is not 1/1 with matlab.  
subplot(2,6,ii);
imshow(log(1+abs(cc)),[-0 0.0001]); brighten(0.4); title(['Ch ' num2str(ii)]);
end

<div class="alert alert-warning">
  <strong><span class="badge">1.4</span></strong> <b>Display k-space data on trajectories</b> 
</div>

In [ ]:
ksp_rss = bart('rss 8', rawdata);
%figure();
lg_ksprss = log(ksp_rss);
lg_ksprss = abs(squeeze(lg_ksprss)); 
dsp = reshape(lg_ksprss,[1 nFE*nSpokes]);


In [ ]:
% This may take some time in Octave... 
%scatter3(trajectory(1, :), trajectory(2,:),trajectory(3,:)[],dsp); colormap('hot');

% Even this one takes a long time 
scatter(trajectory(1, :), trajectory(2,:),[],dsp); colormap('hot');

## 2. Estimate coil sensitivities using BART
***

<div class="alert alert-info">
  <strong><span class="badge">2.1</span></strong> <b>Use BART to estimate coil sensitivities</b> 
</div>

In [ ]:
% Adjoint NUFFT to interpolate data onto cartesian grid. 
% -d denotes dimension (x:y:z, which is 300X300X1)
% -a denotes adjoint transform
% -i denotes inverse transform

adj = bart('nufft -d300:300:1 -i -l -t',trajectory,rawdata);

% Transform regridded image back to k-space 
% 7 is for bitmasking 

adjk = bart('fft -u 7', adj);

% Estimate coil sensitivities using the k-space center (radius of 20). 

calib = bart('ecalib -m1 -I',adjk);


<div class="alert alert-warning">
  <strong><span class="badge">2.2</span></strong> <b>Display sensitivity profiles per channel</b>
</div>


In [ ]:
figure();
for ii=1:12
subplot(3,4,ii);
imagesc(log(1+abs(calib(:,:,1,ii)))); axis off; title(['Ch ' num2str(ii)]);
end

## 3. Reconstruction function 

<div class="alert alert-warning">
  <strong><span class="badge">3.0</span></strong> <b>Use SENSE recon (both cartesian and non-cartesian) provided by BART for demonstration</b> 
</div>

<div class="alert alert-warning">
  <strong><span class="badge">3.0.1</span></strong> <b>Cartesian SENSE (BART) using k-space data obtained by fft of nufft</b> 
</div>

In [ ]:
% Use gridded data for SENSE recon --> BART PICS 
bart_SENSE = bart('pics -l2', adjk, calib);
imshow(abs(bart_SENSE),[])

<div class="alert alert-warning">
  <strong><span class="badge">3.0.2</span></strong> <b>Non-Cartesian SENSE (BART) using provided data</b> 
</div>

In [ ]:
% Use non-cartesian SENSE recon --> BART PICS 
bart_SENSE2 = bart('pics -t',trajectory, rawdata, calib);
imshow(abs(bart_SENSE2),[]);

<div class="alert alert-info">
  <strong><span class="badge">3.1</span></strong> <b>Subsample provided data by factors of 2, 3 and 4.</b>
</div>


In [ ]:
% Please see HelperFunctions folder for subSample.m 

[outRD_x2, outTR_x2] = subSample(rawdata,trajectory,2,nSpokes);

[outRD_x3, outTR_x3] = subSample(rawdata,trajectory,3,nSpokes);

[outRD_x4, outTR_x4] = subSample(rawdata,trajectory,4,nSpokes);

<div class="alert alert-warning">
  <strong><span class="badge">3.2</span></strong> <b>Use non-cartesian SENSE recon (BART) to observe the effects of subsampling</b>
</div>


In [ ]:
% BART non-cartesian sense outputs 
bart_SENSE2_x2 = bart('pics -t',outTR_x2, outRD_x2, calib);
bart_SENSE2_x3 = bart('pics -t',outTR_x3, outRD_x3, calib);
bart_SENSE2_x4 = bart('pics -t',outTR_x4, outRD_x4, calib);

In [ ]:
figure();
subplot(2,2,1); imshow(abs(bart_SENSE2),[]); title('Original');
subplot(2,2,2); imshow(abs(bart_SENSE2_x2),[]); brighten(0.5); title('Undersample X2');
subplot(2,2,3); imshow(abs(bart_SENSE2_x3),[]); brighten(0.5); title('Undersample X3');
subplot(2,2,4); imshow(abs(bart_SENSE2_x4),[]); brighten(0.5); title('Undersample X4');


<p style="color:navy;font-size:16px; background-color:powderblue; border-left: 6px solid navy; display: inline-block"> &nbsp; <i class="fa fa-info-circle fa-1x" style="color:darkblue"></i>  <b>Reminder: Figure-1 from the paper&nbsp;</b></p> <br>

<br>

<div class="row">
<div class="col-sm-5" style="padding-right:5px">
<img src="https://wol-prod-cdn.literatumonline.com/cms/attachment/2594c4ea-88e5-4078-b0ea-9893a6554853/mfig001.jpg" style="width:500px;height:400px" align="left">
</div>
<div class="col-sm-7" style="background-color:powderblue">
<p style="font-size:12px; padding-left:15px"> <b>Implementation of iterative image reconstruction.</b><br><br> Conjugate gradient (CG) iteration is controlled by the central CG process. It is initialized by MR data originating from N receiver channels (1,2,3,…N), acquired with an arbitrary k‐space trajectory. <br> Separately for each channel, these data undergo processing similar to conventional gridding reconstruction, i.e., sampling density correction (D), and resampling along a Cartesian grid, followed by FFT (FT1). The resulting images are individually multiplied by complex conjugate coil sensitivity and summed. After subsequent intensity correction (I), the sum image represents the vector a as defined by Eq. [25]. After initialization with a, the CG process iteratively calculates a progression of images, which converges towards exact reconstruction. For each iteration step, a current residuum image vector needs to be multiplied by the matrix IEH DEI. This is performed by the loop which starts from the CG box. After initial intensity correction (I), the processing is continued separately for each receiver coil. First, the intensity corrected residuum image is multiplied by individual coil sensitivity (Si). The results are transformed into k‐space by FFT and resampled along the experimental k‐space trajectory (FT2), resulting in a set of multiple‐coil k‐space data similar to that obtained experimentally. The following steps are equivalent to those carried out with the original data, yielding an intermediate image, which is fed back into the CG process. Here a refined approximate solution is calculated, which serves for further refinement by continued iteration. <br>As soon as the current approximation is sufficiently accurate, it is output and undergoes final intensity correction and k‐space filtering.</p>
</div>
</div>


<p style="color:navy;font-size:16px; background-color:powderblue; border-left: 6px solid navy; display:inline-block"> &nbsp; <i class="fa fa-info-circle fa-1x" style="color:darkblue"></i>  <b> Implementation notes &nbsp; </b></p>

TO BE EDITED

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;FT1&nbsp; </p> <b>&</b> <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;D&nbsp; </p> <b>operations are implemented using BART's NUFFT:</b> <br>

```python 
# -d denotes dimensions in x:y:z
# -i denotes inverse transform

transformed = bart('nufft -d x:y:z -i',trajectory,rawdata); 

```

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;S<sub>$\gamma$</sub>&nbsp; </p> <b> coil sensitivities are estimated using BART's CALDIR. See section 2.1</b>
    
```python
# 20 denotes calibration region size
# Conjugate transform is simply performed using Octave's conj function

S = bart('caldir 20',kspace); 
```

* <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;FT2&nbsp; </p> <b>is implemented using BART's NUFFT:</b> <br>

```python
Output = bart('nufft',trajectory,rawdata); 
```

* <p style="border:1px;border-style:solid;display:inline-block; border-radius:50%"> &nbsp;Sum&nbsp; </p> <b>is implemented using BART's RRS:</b> <br>

```python
Sum = bart('rss bitmask',input); 
```


<div class="alert alert-info">
  <strong><span class="badge">3.2</span></strong> <b>Calculate density compensation factor for operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;D&nbsp; </p> </b>
</div>

This operation may take approximately 50 seconds. A simpler method could have been applied here, yet would like to use Brian's MEXified implementations. 

In [ ]:
dcf = calcdcflut(trajectory,300); 
dcf = reshape(dcf,[3 nFE nSpokes]);

<div class="alert alert-info">
  <strong><span class="badge">3.3</span></strong> <b>Calculate intensity correction matrix for operatoin <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;I&nbsp; </p></b>
</div>


In [ ]:
# Root sum of square of sensitivities from 12 channels 

I = real(bart('rss 8',calib.*conj(calib)));
imagesc(I);

<div class="alert alert-info">
  <strong><span class="badge">3.4</span></strong> <b>Define operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;E&nbsp; </p></b>
</div>


In [ ]:
function E = opE(inp,S,traj,I)
% Here inp size is (300,300)
% S is the sensitivities from 12 channels (300,300,1,12)
% traj is the k-space coordinates
% Normally, I is not a part of operation E, added here for convenience. 

inp = inp.*I; % Intensity corr

tmp = S.*inp; % Multiply intensity corrected images with coil sensitivities

E = bart('nufft',traj,tmp); % Transform back to the non-uniform k-space

end

<div class="alert alert-info">
  <strong><span class="badge">3.5</span></strong> <b>Define operation <p style="border:1px;border-style:solid;display:inline-block"> &nbsp;E<sup>H</sup>&nbsp; </p></b>
</div>


In [ ]:
function out = opEH(dcf,inp,S,traj,I)
% dcf is density compensation factor , see 3.2
% input samples respecting the size of the rawdata (1,512,96,12) 
% S is the coil sensitivities 
% traj is the k-space coordinates 

  % Inverse NUFFT to the image domain using Toeplitz embedding
  tmp = bart('nufft -d300:300:1 -i -t',traj,inp.*sqrt(dcf(1,:,:)));
  
  % Get complex conjugate of sensitivity profiles
  Sstar = conj(S(:,:,1,:));
  
  % Root sum of square of the images, scaled by complex conjugate of the sensitivity maps.
  tmp2 = bart('rss 8 ',tmp.*Sstar);
  
  % Intensity correction
  out = tmp2.*I;

end

In [ ]:
function b = cg_solve(a,I,S,dcf,maxstep,trajectory,usrthr)
p = a;
r = a;
b = zeros(300,300);

for ii=1:maxstep

disp(['Iteration -->' num2str(ii)]);

thr = norm(r).^2./norm(b).^2;
disp(thr);

%if all(logical(thr(:)<usrthr))
 %   break;
%end

E = opE(p,S,trajectory);

q = opEH(dcf,E,S,trajectory,I);

term = abs(dot(r,r))/dot(p,q);
% The following fails 
% term = real(r*ctranspose(r)/ctranspose(p)*q);
b = b + term*p;

rprev = r;
r = r - term*q;

term2 = abs(dot(r,r)/dot(rprev,rprev));

% The following fails 
% term2 = real(r*ctranspose(r)/ctranspose(rprev)*rprev);

p = r + term2*p;

end

end

In [ ]:
a = opEH(dcf,rawdata,calib,trajectory,I);
b = cg_solve(a,I,calib,dcf,10,trajectory,0.001);

In [ ]:
imagesc(abs(b(30:300,40:250)))

In [ ]:
imagesc(b.*I);

In [ ]:
k_b = fftshift(fft2(b));

k = 0.5 + 1/pi.*atan(100.*((40-abs(-150:149))/40));
yy = repmat(k,[300 1]);

lele= ifft2(ifftshift(k_b.*yy'));

In [ ]:
imshow(abs(lele),[])